In [ ]:
!apt-get update
!apt-get install ffmpeg -y

In [7]:
import concurrent.futures
import os
import time
import subprocess
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Create a list to store video entries
videos = []

# Define widgets
url_input = widgets.Text(
    description='URL:',
    placeholder='Enter M3U8 URL',
    layout=widgets.Layout(width='80%')
)

output_input = widgets.Text(
    description='Output:',
    placeholder='output_file.mp4',
    layout=widgets.Layout(width='80%')
)

add_button = widgets.Button(
    description='Add Video',
    button_style='success',
    icon='plus'
)

download_button = widgets.Button(
    description='Download All',
    button_style='primary',
    icon='download'
)

clear_button = widgets.Button(
    description='Clear All',
    button_style='danger',
    icon='trash'
)

max_workers_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=5,
    step=1,
    description='Max Parallel:',
    layout=widgets.Layout(width='50%')
)

video_list = widgets.Output()

status_output = widgets.Output()

def check_ffmpeg():
    """Check if ffmpeg is installed, if not install it"""
    try:
        subprocess.check_output('which ffmpeg', shell=True)
        return True
    except subprocess.CalledProcessError:
        print("Installing ffmpeg...")
        subprocess.run('apt-get update && apt-get install -y ffmpeg', shell=True)
        return True

def add_video(b):
    """Add a video to the list"""
    url = url_input.value.strip()
    output = output_input.value.strip()

    if not url:
        with status_output:
            clear_output()
            print("Please enter a URL")
        return

    if not output:
        # Generate a default output name if none provided
        output = f"video_{len(videos)+1}.mp4"

    # Add .mp4 extension if not present
    if not output.lower().endswith('.mp4'):
        output = output + '.mp4'

    videos.append({
        "url": url,
        "output": output
    })

    # Clear inputs
    url_input.value = ''
    output_input.value = ''

    # Update list display
    update_video_list()

    with status_output:
        clear_output()
        print(f"Added: {output}")

def update_video_list():
    """Update the displayed list of videos"""
    with video_list:
        clear_output()
        if not videos:
            print("No videos added yet.")
        else:
            print("Videos to download:")
            for i, video in enumerate(videos):
                print(f"{i+1}. {video['output']} - {video['url'][:50]}...")

                # Add remove button for each entry
                remove_btn = widgets.Button(
                    description=f'Remove #{i+1}',
                    button_style='warning',
                    layout=widgets.Layout(width='150px')
                )
                remove_btn.index = i
                remove_btn.on_click(lambda b: remove_video(b.index))
                display(remove_btn)
                print()

def remove_video(index):
    """Remove a video from the list"""
    if 0 <= index < len(videos):
        removed = videos.pop(index)
        update_video_list()
        with status_output:
            clear_output()
            print(f"Removed: {removed['output']}")

def clear_videos(b):
    """Clear all videos from the list"""
    videos.clear()
    update_video_list()
    with status_output:
        clear_output()
        print("All videos cleared")

def download_video(video):
    """Download a single video and show progress"""
    # Get video duration first
    duration_cmd = f'ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 "{video["url"]}"'
    try:
        duration = float(subprocess.check_output(duration_cmd, shell=True).decode('utf-8').strip())
    except:
        duration = 0  # If we can't get duration, we'll use a time-based progress bar

    # Create progress bar
    pbar = tqdm(total=100, desc=f"Downloading {video['output']}", unit="%")

    # Prepare ffmpeg command with progress output
    cmd = f'ffmpeg -y -i "{video["url"]}" -c copy -progress - "{video["output"]}" 2>/dev/null'

    # Start process
    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)

    # Variables to track progress
    progress = 0
    current_time = 0

    # Monitor progress
    if duration > 0:
        # Duration-based progress if we could get the duration
        for line in process.stdout:
            if 'out_time_ms=' in line:
                time_ms = int(line.strip().split('=')[1])
                current_time = time_ms / 1000000  # convert to seconds
                progress = min(int(current_time / duration * 100), 100)
                pbar.update(progress - pbar.n)  # Update to current progress
    else:
        # Time-based progress (less accurate)
        start_time = time.time()
        while process.poll() is None:
            time.sleep(1)
            elapsed = time.time() - start_time
            # Assume average video is around 60 minutes max
            progress = min(int(elapsed / 3600 * 100), 100)
            pbar.update(progress - pbar.n)

    # Make sure we reach 100%
    pbar.update(100 - pbar.n)
    pbar.close()

    return video["output"]

def start_downloads(b):
    """Start downloading all videos in parallel"""
    if not videos:
        with status_output:
            clear_output()
            print("No videos to download. Please add videos first.")
        return

    # Make sure ffmpeg is installed
    check_ffmpeg()

    with status_output:
        clear_output()
        print(f"Starting download of {len(videos)} videos with {max_workers_slider.value} parallel workers...")

    # Download videos in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers_slider.value) as executor:
        futures = [executor.submit(download_video, v) for v in videos]

        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()
                with status_output:
                    print(f"✓ Completed: {result}")
            except Exception as e:
                with status_output:
                    print(f"✗ Error: {str(e)}")

    # Final status update
    with status_output:
        print("All downloads completed!")
        print("\nDownloaded files:")
        for i, video in enumerate(videos):
            if os.path.exists(video['output']):
                size_mb = os.path.getsize(video['output']) / (1024 * 1024)
                print(f"{i+1}. {video['output']} ({size_mb:.2f} MB)")
            else:
                print(f"{i+1}. {video['output']} (Download failed)")

# Connect button actions
add_button.on_click(add_video)
download_button.on_click(start_downloads)
clear_button.on_click(clear_videos)

# Create the UI layout
header = widgets.HTML(value="<h2>M3U8 Video Downloader</h2>")
subtitle = widgets.HTML(value="<p>Download videos from M3U8 playlists using ffmpeg</p>")

input_box = widgets.VBox([
    widgets.HBox([url_input]),
    widgets.HBox([output_input]),
    widgets.HBox([add_button, download_button, clear_button]),
    widgets.HBox([max_workers_slider])
])

# Final layout assembly
main_layout = widgets.VBox([
    header,
    subtitle,
    input_box,
    widgets.HTML(value="<hr>"),
    widgets.HTML(value="<h3>Video Queue:</h3>"),
    video_list,
    widgets.HTML(value="<hr>"),
    widgets.HTML(value="<h3>Status:</h3>"),
    status_output
])

# Display the UI
display(main_layout)

# Initialize the list display
update_video_list()

# Display initial instructions in status area
with status_output:
    print("Ready! Enter M3U8 URL and output filename, then click 'Add Video'.")
    print("After adding all videos, click 'Download All' to begin.")